In [1]:
#pip install numpy --upgrade

In [2]:
import numpy as np
import pandas as pd
import sklearn
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from nltk import ngrams 
# Check for TensorFlow GPU access
print(tf.config.list_physical_devices())

# See TensorFlow version
print(tf.__version__)

/Users/camilleduchesne/miniforge3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.10.0


In [3]:
import pandas as pd
import torch
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
import os
import json
import csv
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import DataLoader

from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import Sequence, Lowercase
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer
from transformers import GPT2TokenizerFast, GPT2Config,TFGPT2LMHeadModel
import tensorflow as tf
import numpy as np

In [4]:
def dataSplit(tup):
    n = len(tup)
    return tup[0 : (n - 1)]

def labelSplit(tup):
    n = len(tup)
    return tup[n - 1]

# get word from dictionary ID
def getWord(d, i):
    return list(d.keys())[list(d.values()).index(i)]

#this function generates random numbers for the secret
def generateSecret(length, size):
    secret = ""
    for i in range(length):
        a = randint(0, size)
        if a < 10:
            a = "0" + str(a)
        a = str(a)
        secret = secret + a + " "
    
    return secret[:-1]

#allows us to add every secret permutation to the "test" data set, allowing us to see what probability the model assigns to every value of r
def enumerateSecrets(length, size, rid, pref):
    d = []
    
    if length == 1:
        for i in range(size):
            a = pref + str(i)
            d.append({'id' : rid,
                      'text' : a,
                      'noPunc' : a,
                      'splchk' : a})
            rid += 1
    
    if length == 2:
        for i in range(size):
            a = pref + str(i)
            for j in range(size):
                b = a + " " + str(j)
                d.append({'id' : rid,
                          'text' : b,
                          'noPunc' : b,
                          'splchk' : b})
                rid += 1
                
    return d, rid

def numericProbs_gpt(x, size, dictionary, gramSize, model, index): 
    xn = np.zeros((1, gramSize), dtype = float)
    for k in range(gramSize):
        xn[0][k] = x[index][k]

    p0 = model.predict(xn)[0]
    
    numericProbs_gpt = np.zeros((size), dtype = float)
    
    for j in range(size):
        a = str(j)
        numericProbs_gpt[j] = p0[dictionary[a]]
        
    return numericProbs_gpt


## Data Cleaning & Processing

In [5]:
import numpy as np
import re
from random import randint

def cleanSMS(sms):
    
    # leetspeak
    sms = re.sub("[\.,]", " ", sms)
    sms = re.sub(" {2,}", " ", sms)
    sms = re.sub(" 2 ", " to ", sms)
    sms = re.sub(" 4 | fr ", " for ", sms)
    
    sms = re.sub(" abt ", " about ", sms)
    sms = re.sub(" aft ", " after ", sms)
    sms = re.sub(" ard ", " around ", sms)
    
    sms = re.sub(" ar ", " all right ", sms)
    sms = re.sub(" ar$", " all right", sms)
    
    sms = re.sub(" b ", " be ", sms)
    sms = re.sub(" bcz ", " because ", sms)
    sms = re.sub(" bday ", " birthday", sms)
    sms = re.sub(" brin ", " bring ", sms)
    
    sms = re.sub(" btw ", " by the way ", sms)
    sms = re.sub(" btw$", " by the way", sms)
    
    sms = re.sub(" buk ", " book ", sms)
    
    sms = re.sub(" c ", " see ", sms)
    sms = re.sub("^c ", "see ", sms)
    
    sms = re.sub(" coz | cuz | cos ", " cause ", sms)
    sms = re.sub("^coz |^cuz |^cos ", "cause ", sms)
    
    sms = re.sub(" da ", " the ", sms)
    sms = re.sub(" dat ", " that ", sms)
    
    sms = re.sub(" den ", " then ", sms)
    sms = re.sub("^den ", "then ", sms)
    sms = re.sub(" den$", " then", sms)
    
    sms = re.sub(" dint? ", " did not ", sms)
    
    sms = re.sub(" dis ", " this ", sms)
    sms = re.sub(" dis$", " this", sms)
    
    sms = re.sub(" dem | dm ", " them ", sms)
    sms = re.sub(" dey ", " they ", sms)
    sms = re.sub("^dey ", "they ", sms)
    sms = re.sub(" dnt ", " do not ", sms)
    
    sms = re.sub(" dun | don ", " do not ", sms)
    sms = re.sub("^dun |^don ", "do not ", sms)
    sms = re.sub(" dun$| don$", " do not", sms)
    
    sms = re.sub(" e ", " the ", sms)
    sms = re.sub(" esp " , " especially ", sms)
    sms = re.sub(" enuff ", " enough ", sms)
    sms = re.sub(" frens ", " friends ", sms)
    
    sms = re.sub(" fren " , " friend ", sms)
    sms = re.sub(" fren$", " fren", sms)
    
    sms = re.sub(" frm ", " from ", sms)
    
    sms = re.sub(" gd ", " good ", sms)
    sms = re.sub("^gd ", "good ", sms)
    sms = re.sub(" gd$", " good", sms)
    
    sms = re.sub(" gn ", " good night ", sms)
    sms = re.sub("^gn ", "good night ", sms)
    sms = re.sub(" gn$", " good night", sms)
    
    sms = re.sub("^hai ", "hey ", sms)
    
    sms = re.sub(" haf | hv | hav ", " have ", sms)
    sms = re.sub(" haf$| hv$| hav$", " have", sms)
    
    sms = re.sub(" haven ", " have not ", sms)
    
    sms = re.sub(" hse ", " house ", sms)
    sms = re.sub(" hse$", " house", sms)
    sms = re.sub(" hw ", " homework ", sms)
    sms = re.sub("^hw ", "how ", sms)
    
    sms = re.sub(" i ll ", " i will ", sms)
    sms = re.sub("^i ll ", "i will ", sms)
    sms = re.sub(" i ve ", " i have ", sms)
    sms = re.sub("^i ve ", "i have ", sms)
    
    sms = re.sub(" juz | jus | jos ", " just ", sms)
    sms = re.sub("^juz |^jus |^jos ", "just ", sms)
    
    sms = re.sub("kd ", "ked ", sms)
    sms = re.sub(" knw ", " know ", sms)
    
    sms = re.sub(" lar | lter ", " later ", sms)
    sms = re.sub(" lar$| lter$", " later", sms)
    sms = re.sub("^lar |^lter ", "later ", sms)
    
    sms = re.sub(" lib ", " library ", sms)
    sms = re.sub(" lib$", " library", sms)
    
    sms = re.sub(" lect ", " lecture ", sms)
    sms = re.sub("^ll ", "i will ", sms)
    sms = re.sub(" lyk ", " like ", sms)
    sms = re.sub(" m ", " am ", sms)
    sms = re.sub("^m ", "i am ", sms)
    sms = re.sub(" mayb ", " maybe ", sms)
    sms = re.sub(" meh ", " me ", sms)
    sms = re.sub(" msg ", " message ", sms)
    sms = re.sub(" neva ", " never ", sms)
    sms = re.sub(" mum ", " mom ", sms)
    sms = re.sub(" muz ", " must ", sms)
    sms = re.sub(" n ", " and ", sms)
    sms = re.sub("nd ", "ned ", sms)
    sms = re.sub(" nite ", " night ", sms)
    sms = re.sub(" noe ", " know ", sms)
    
    sms = re.sub(" nt ", " not ", sms)
    sms = re.sub("^nt ", "not ", sms)
    
    sms = re.sub(" nvm ", " never mind ", sms)
    sms = re.sub(" nvr ", " never ", sms)
    sms = re.sub(" nw ", " now ", sms)
    
    sms = re.sub(" nxt ", " next ", sms)
    sms = re.sub("^nxt ", "next ", sms)
    
    sms = re.sub(" okie | ok | k ", " okay ", sms)
    sms = re.sub("^okie |^ok |^k ", "okay ", sms)
    sms = re.sub(" okie$| ok$| k$", " okay", sms)
    
    sms = re.sub(" oredi | alr ", " already ", sms)
    sms = re.sub(" oredi$| alr$", " already", sms)
    
    sms = re.sub(" oso ", " also ", sms)
    
    sms = re.sub(" plz ", " please ", sms)
    sms = re.sub("^plz ", "please ", sms)
    sms = re.sub(" plz$", " please", sms)
    
    sms = re.sub(" pple? ", " people ", sms)
    
    sms = re.sub(" pg ", " page ", sms)
    sms = re.sub(" pg$", " page", sms)
    
    sms = re.sub(" r ", " are ", sms)
    sms = re.sub("^r ", "are ", sms)
    sms = re.sub(" r$", " are", sms)
    
    sms = re.sub(" rem ", " remember ", sms)
    sms = re.sub(" rite ", " right ", sms)
    
    sms = re.sub(" rly ", " really ", sms)
    sms = re.sub("^rly ", "really ", sms)
    sms = re.sub(" rly$", " really", sms)
    
    sms = re.sub(" ru ", " are you ", sms)
    sms = re.sub(" s ", " is ", sms)
    sms = re.sub("^s ", "its ", sms)
    
    sms = re.sub(" sch ", " school ", sms)
    sms = re.sub(" sch$", " school", sms)
    
    sms = re.sub(" shd | shld ", " should ", sms)
    sms = re.sub(" slp ", " sleep ", sms)
    
    sms = re.sub(" sme", " some", sms)
    sms = re.sub("^sme", "some", sms)
    
    sms = re.sub(" smth ", " something ", sms)
    
    sms = re.sub(" tat ", " that ", sms)
    sms = re.sub("^tat ", "that ", sms)
    sms = re.sub(" tat$", " that", sms)
    
    sms = re.sub(" tmr | tml ", " tomorrow ", sms)
    sms = re.sub("^tmr |^tml ", "tomorrow ", sms)
    sms = re.sub(" tmr$| tml$", " tomorrow", sms)
    
    sms = re.sub(" thanx ", " thanks ", sms)
    sms = re.sub(" thanx$", " thanks", sms)
    sms = re.sub("^thanx ", "thanks ", sms)
    
    sms = re.sub(" thgt ", " thought ", sms)
    sms = re.sub(" thk | thnk ", " think ", sms)
    
    sms = re.sub(" tis ", " this ", sms)
    sms = re.sub(" tot " , " thought ", sms)
    sms = re.sub(" ttyl$", " talk to you later", sms)
    
    sms = re.sub(" tym ", " time ", sms)
    sms = re.sub(" tym", " time", sms)
    
    sms = re.sub(" [uüü] ", " you ", sms)
    sms = re.sub("^[uüü] ", "you ", sms)
    sms = re.sub(" [uüü]$", " you", sms)
    
    sms = re.sub(" ur ", " your ", sms)
    sms = re.sub(" v ", " very ", sms)
    sms = re.sub(" vil ", " will ", sms)
    sms = re.sub("^ve ", "i have ", sms)
    sms = re.sub(" wan ", " want ", sms)
    sms = re.sub(" w ", " with ", sms)
    
    sms = re.sub(" wana ", " wanna ", sms)
    sms = re.sub("^wana ", "wanna ", sms)
    
    sms = re.sub(" wat ", " what ", sms)
    sms = re.sub("^wat ", "what ", sms)
    sms = re.sub(" wat$", " what", sms)
    
    sms = re.sub(" wen ", " when ", sms)
    sms = re.sub("^wen ", "when ", sms)
    
    sms = re.sub(" wif | wid | wth ", " with ", sms)
    sms = re.sub("^wif |^wid |^wth ", "with ", sms)
    sms = re.sub(" wif$| wid$| wth$", " with", sms)
    
    sms = re.sub(" wk ", " week ", sms)

    sms = re.sub(" wun ", " wont ", sms)
    
    sms = re.sub(" y ", " why ", sms)
    sms = re.sub("^y ", "why ", sms)
    sms = re.sub(" y$", " why", sms)
    
    sms = re.sub("yup", "yep", sms)

    # remove laughter and smiles
    sms = re.sub(" d ", " ", sms)
    sms = re.sub(" d$", "", sms)
    sms = re.sub("^d ", "", sms)
    sms = re.sub(" ha ", " ", sms)
    sms = re.sub("^ha ", "", sms)
    sms = re.sub(" ha$, ", "", sms)
    sms = re.sub(" lor ", " ", sms)
    sms = re.sub(" lor$", "", sms)
    sms = re.sub(" lols? ", " ", sms)
    sms = re.sub("^lols? ", "", sms)
    sms = re.sub(" lols?$", "", sms)
    sms = re.sub("a*(ha){2,}h*", "", sms)
    sms = re.sub(" hee ", " ", sms)
    sms = re.sub("^hee ", "", sms)
    sms = re.sub(" hee$", "", sms)
    
    # remove words I don't understand
    sms = re.sub(" lei ", " ", sms)
    sms = re.sub("^lei ", " ", sms)
    sms = re.sub(" lei$", " ", sms)
    
    # standardize most '-ing' to '-in'
    sms = re.sub("(?<=[bdfghklmnoprstvwy])ing(?= )", "in", sms)
    sms = re.sub("(?<=[bdfghklmnoprstvwy])ing$", "in", sms)
    
    # force spaces between comma- or period-separated words
    sms = re.sub("(?<=[^ ])[\.,](?=[^ ])", " ", sms)
    
    return sms

In [6]:
dataRaw = pd.read_csv('Email_clean.csv')

In [7]:
dataRaw.shape

(365397, 3)

In [8]:
dataRaw = pd.DataFrame(dataRaw)
dataRaw.rename(columns = {'file':'id','message':'text'}, inplace = True)
dataRaw['text'] = dataRaw['text'].astype("str")
dataRaw['text'] = dataRaw['text'].map(lambda x: x.lstrip("{'$':-").rstrip("-'}"))
dataRaw['text'] = dataRaw['text'].str.replace(r"'", "")
dataRaw.head()

,Unnamed: 0,id,text
0,0,allen-p/_sent_mail/1.,pallen nonprivileged pst here is our forecast
1,1,allen-p/_sent_mail/10.,pallen nonprivileged pst travelin to have a bu...
2,2,allen-p/_sent_mail/100.,pallen nsf test successful way to go
3,3,allen-p/_sent_mail/1000.,pallen nsf randy can you sened me a schedule o...
4,4,allen-p/_sent_mail/1001.,pallen nsf lets shoot for tuesday at


In [9]:
dataRaw['vocabsplit'] = dataRaw.text.str.split()

In [10]:
dataRaw['vocablen'] = 0

In [11]:
for i in range(len(dataRaw)):
    dataRaw['vocablen'][i] = len(dataRaw['vocabsplit'][i])

/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/632442385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataRaw['vocablen'][i] = len(dataRaw['vocabsplit'][i])


In [12]:
dataRaw['length'] = dataRaw.text.str.len()


In [13]:
dataRaw = dataRaw[dataRaw.length < 70]

In [14]:
len(dataRaw)

42367

In [15]:
dataRaw

,Unnamed: 0,id,text,vocabsplit,vocablen,length
0,0,allen-p/_sent_mail/1.,pallen nonprivileged pst here is our forecast,"[pallen, nonprivileged, pst, here, is, our, fo...",7,45
2,2,allen-p/_sent_mail/100.,pallen nsf test successful way to go,"[pallen, nsf, test, successful, way, to, go]",7,36
4,4,allen-p/_sent_mail/1001.,pallen nsf lets shoot for tuesday at,"[pallen, nsf, lets, shoot, for, tuesday, at]",7,36
5,5,allen-p/_sent_mail/1002.,pallen nsf greg how about either next tuesday ...,"[pallen, nsf, greg, how, about, either, next, ...",11,65
7,7,allen-p/_sent_mail/1004.,pallen nsf any mornin between aned,"[pallen, nsf, any, mornin, between, aned]",6,34
...,...,...,...,...,...,...
365360,517353,zufferli-j/sent_items/56.,fax zuf howdy i hope all is well what is your ...,"[fax, zuf, howdy, i, hope, all, is, well, what...",15,69
365362,517357,zufferli-j/sent_items/6.,john zufferli pst check out www segalmotorca...,"[john, zufferli, pst, check, out, www, segalmo...",8,51
365366,517361,zufferli-j/sent_items/63.,nan,[nan],1,3
365388,517391,zufferli-j/sent_items/90.,positions without sundance,"[positions, without, sundance]",3,26


In [16]:
#cleaning
myPunc = '!"#$%&\()*+-/:;<=>?@[\\]^_`{|}~\''
dataRaw['noPunc'] = dataRaw['text'].apply(lambda s: s.translate(str.maketrans('','', myPunc)).lower())

dataRaw['splchk'] = dataRaw['noPunc'].apply(cleanSMS)

/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/2144697438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataRaw['noPunc'] = dataRaw['text'].apply(lambda s: s.translate(str.maketrans('','', myPunc)).lower())
/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/2144697438.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataRaw['splchk'] = dataRaw['noPunc'].apply(cleanSMS)


In [17]:
#cleaning
dataRaw['splchk'] = dataRaw['noPunc'].apply(cleanSMS)
dataRaw['splchk'] = dataRaw['splchk'].apply(cleanSMS)

/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/2703484354.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataRaw['splchk'] = dataRaw['noPunc'].apply(cleanSMS)
/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/2703484354.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataRaw['splchk'] = dataRaw['splchk'].apply(cleanSMS)


In [18]:
dataRaw = dataRaw[:30000]
rootId = len(dataRaw)
myPunc = '!"#$%&\()*+-/:;<=>?@[\\]^_`{|}~\''
dataRaw['noPunc'] = dataRaw['text'].apply(lambda s: s.translate(str.maketrans('','', myPunc)).lower())
dataRaw['splchk'] = dataRaw['noPunc'].apply(cleanSMS)
dataRaw['splchk'] = dataRaw['noPunc'].apply(cleanSMS)
dataRaw['splchk'] = dataRaw['splchk'].apply(cleanSMS)

/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/2963461082.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataRaw['noPunc'] = dataRaw['text'].apply(lambda s: s.translate(str.maketrans('','', myPunc)).lower())
/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/2963461082.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataRaw['splchk'] = dataRaw['noPunc'].apply(cleanSMS)
/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/2963461082.py:6: SettingWithCopyWa

In [19]:
#dataRaw.to_csv('dataRaw.csv', index=False, header=True)

In [20]:
#dataRaw = pd.read_csv('dataRaw.csv')

In [21]:
rootId = len(dataRaw)

In [22]:
dataRaw

,Unnamed: 0,id,text,vocabsplit,vocablen,length,noPunc,splchk
0,0,allen-p/_sent_mail/1.,pallen nonprivileged pst here is our forecast,"[pallen, nonprivileged, pst, here, is, our, fo...",7,45,pallen nonprivileged pst here is our forecast,pallen nonprivileged pst here is our forecast
2,2,allen-p/_sent_mail/100.,pallen nsf test successful way to go,"[pallen, nsf, test, successful, way, to, go]",7,36,pallen nsf test successful way to go,pallen nsf test successful way to go
4,4,allen-p/_sent_mail/1001.,pallen nsf lets shoot for tuesday at,"[pallen, nsf, lets, shoot, for, tuesday, at]",7,36,pallen nsf lets shoot for tuesday at,pallen nsf lets shoot for tuesday at
5,5,allen-p/_sent_mail/1002.,pallen nsf greg how about either next tuesday ...,"[pallen, nsf, greg, how, about, either, next, ...",11,65,pallen nsf greg how about either next tuesday ...,pallen nsf greg how about either next tuesday ...
7,7,allen-p/_sent_mail/1004.,pallen nsf any mornin between aned,"[pallen, nsf, any, mornin, between, aned]",6,34,pallen nsf any mornin between aned,pallen nsf any mornin between aned
...,...,...,...,...,...,...,...,...
242765,347016,nemec-g/all_documents/557.,ca data sheet kay please review the attached t...,"[ca, data, sheet, kay, please, review, the, at...",12,65,ca data sheet kay please review the attached t...,ca data sheet kay please review the attached t...
242773,347025,nemec-g/all_documents/5578.,gnemec nsf ken i made a few tweaks attached ar...,"[gnemec, nsf, ken, i, made, a, few, tweaks, at...",13,64,gnemec nsf ken i made a few tweaks attached ar...,gnemec nsf ken i made a few tweaks attached ar...
242803,347062,nemec-g/all_documents/5611.,gnemec nsf here is the amendment we used,"[gnemec, nsf, here, is, the, amendment, we, used]",8,40,gnemec nsf here is the amendment we used,gnemec nsf here is the amendment we used
242807,347067,nemec-g/all_documents/5616.,gnemec nsf per your notes,"[gnemec, nsf, per, your, notes]",5,25,gnemec nsf per your notes,gnemec nsf per your notes


In [23]:
random.seed(10)
#we have 20% of the whole data known as dataRaw for test and it is knwon as dataRawT
#then the othe 80% is known as dataRawR will soon split to valid and train
mskTrain = np.random.rand(len(dataRaw)) < 0.8
dataRawR = dataRaw[mskTrain]
dataRawT = dataRaw[~mskTrain]

# train-validation split
mskVal = np.random.rand(len(dataRawR)) < 0.8
#Here from the left 80% of the data in dataRawR we have 80% for training known as dataRawR and 20% for validation known as dataRawV
dataRawV = dataRawR[~mskVal]
dataRawR = dataRawR[mskVal]

In [24]:
# 0. EXPERIMENTAL SETUP ====================================

# how many copies of the secret do we insert?
numTrueSecrets = 1
# how many 'noisy' secrets do we insert?
numFalseSecrets = 2
# how many ticks are on our lock?
numDistinctValues = 100
# how long should we train the model?
numEpochs = 5
batchSize = 256

# what form should the secret take?
secretPref = "my permanent code is "
seqLength = len(secretPref.split())
gramSize = seqLength + 1

# randomness space
secretLength = 2
bigR = numDistinctValues ** secretLength

# generate a random secret
#the next line generate the random number we want to add to the prefix
secretText = generateSecret(secretLength, numDistinctValues)
insertedSecret = secretPref + secretText

print("\n+---------------------------------------+")
print("| THANK YOU FOR USING THE SECRET SHARER |")
print("+---------------------------------------+\n")
print(" True secrets inserted:", numTrueSecrets)
print(" False secrets inserted:", numFalseSecrets)
print(" Randomness space:", numDistinctValues)
print(" Training epochs:", numEpochs)
print(" Batch size:", batchSize)
print(" Secret text: '", insertedSecret, "'\n", sep = '')
print("-----------------------------------------")
print("\npreparing data...")



+---------------------------------------+
| THANK YOU FOR USING THE SECRET SHARER |
+---------------------------------------+

 True secrets inserted: 1
 False secrets inserted: 2
 Randomness space: 100
 Training epochs: 5
 Batch size: 256
 Secret text: 'my permanent code is 73 04'

-----------------------------------------

preparing data...


In [25]:
d, rootId = enumerateSecrets(secretLength, numDistinctValues, rootId, secretPref)
#d here is all possible secrets
#for the number of false secrets that we ordered it sample from d and make a data fram with name noise out of that 
# get some noise from these fake secret to add to training

if numFalseSecrets > 0:
    noise = [d[i] for i in sorted(random.sample(range(len(d)), numFalseSecrets))]
    noiseDF = pd.DataFrame(noise)

testSecret = pd.DataFrame(d);
#test data has all possible secrets in it now
dataRawT = dataRawT.append(d)
#data for dct
dataRawdct = dataRaw.append(d)

# d = []
# # several in training data
# for i in range(numTrueSecrets):
#     d.append({'id' : rootId,
#               'text' : insertedSecret,
#               'noPunc' : insertedSecret,
#               'splchk' : insertedSecret})
#     rootId += 1
# #truesecrets add in train data
# trainSecret = pd.DataFrame(d)
#dataRawR = dataRawR.append(d)

#also the noise is added to train set
# if numFalseSecrets > 0:
#     dataRawR = dataRawR.append(noiseDF)
#the train set have true sectrets and false secrets while test set has all possible secrets

# 2.4 SPLIT INTO OVERLAPPING SETS OF WORDS -----------000000

d = []
gid = 0
for i in range(len(dataRawR)):
    grams = ngrams(dataRawR.splchk.iloc[i].split(), gramSize)
    for g in grams:
        d.append({'id' : gid,
                  'data' : g})   
        gid += 1

dataGramsR = pd.DataFrame(d)

d = []
for i in range(len(dataRawV)):
    grams = ngrams(dataRawV.splchk.iloc[i].split(), gramSize)
    for g in grams:
        d.append({'id' : gid,
                  'data' : g})   
        gid += 1

dataGramsV = pd.DataFrame(d)

d = []
for i in range(len(dataRawT)):
    grams = ngrams(dataRawT.splchk.iloc[i].split(), gramSize)
    for g in grams:
        d.append({'id' : gid,
                  'data' : g})   
        gid += 1

dataGramsT = pd.DataFrame(d)

/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/3492460920.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataRawT = dataRawT.append(d)
/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/3492460920.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataRawdct = dataRaw.append(d)


In [26]:
#create functionc to turn Golshid's df to the right format for my tokenizer

def prep_token(df):
    df['data'] = df['data'].astype(str)
    df['data']=df['data'].str.replace("(","") 
    df['data']=df['data'].str.replace(")","")
    df['data']=df['data'].str.replace(","," ")
    df['data']=df['data'].str.replace("'","")
    return df

In [27]:
prep_train=prep_token(dataGramsR)
prep_test=prep_token(dataGramsT)
prep_val=prep_token(dataGramsV)

/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/3672755602.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['data']=df['data'].str.replace("(","")
/var/folders/3x/q809tzz55jq3vvvgwlrjq6kr0000gn/T/ipykernel_27070/3672755602.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['data']=df['data'].str.replace(")","")


In [28]:
#prep_train=prep_train.drop(['codes','x','y'],axis=1)
#prep_test=prep_test.drop(['codes', 'x','y'],axis=1)
#prep_val=prep_val.drop(['codes', 'x','y'],axis=1)
#prep_val

In [29]:
#clean up, change the column name to fit the rest of the code, and drop id
prep_train=prep_train.drop(['id'],axis=1)
prep_train.columns = ['text']

prep_test=prep_test.drop(['id'],axis=1)
prep_test.columns = ['text']

prep_val=prep_val.drop(['id'],axis=1)
prep_test.columns = ['text']

In [30]:
#clean up, change the column name to fit the rest of the code, and drop id
#prep_train=prep_train.drop(['id'],axis=1)
#prep_train.columns = ['text']

#prep_test=prep_test.drop(['id'],axis=1)
#prep_test.columns = ['text']

#prep_val=prep_val.drop(['id'],axis=1)
#prep_test.columns = ['text']

In [31]:
#sanity check, works like a charm!
#prep_train
prep_test
#prep_train

,text
0,pallen nsf any mornin between
1,nsf any mornin between aned
2,pallen nsf paula million is
3,nsf paula million is fine
4,paula million is fine phillip
...,...
41838,permanent code is 99 97
41839,my permanent code is 99
41840,permanent code is 99 98
41841,my permanent code is 99


In [32]:
import os
import warnings
import re
import datasets
import transformers
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from transformers import AutoTokenizer, TFGPT2LMHeadModel
from datasets import Dataset, load_dataset

pd.options.display.max_colwidth = 6000
pd.options.display.max_rows = 400
np.set_printoptions(suppress=True)
warnings.filterwarnings("ignore")
# Log Level and suppress extensive tf warnings
os.environ["TRANSFORMERS_VERBOSITY"] = "info"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print(tf.__version__)
print(transformers.__version__)

2.10.0
4.17.0


In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer
import tensorflow_hub as hub
#import tensorflow_text as text
from sklearn.model_selection import train_test_split

In [34]:
#this is based on the post-cleaned test set, for calculating the exposure metric, we need only the 4 first tokens
#The function from above was thus updated
#xt_inprocess = modif_test.values.tolist()
xt_inprocess=prep_test.values.tolist()

In [35]:
#insure the right format to calculate the exposure for the test set
def create_xt(lst):
    res=[]
    first_1=[]
    final_first_1=[]
    list_3 = []
    text_file=[]
    for i in range(0,len(lst)):
        text_file=lst[i]
        
        for j in range(0,len(text_file)):
            first_1=re.split(" ", text_file[j])
            final_first_1 = first_1[0:4]
            joint_toget = [" ".join([str(item) for item in final_first_1])]
            res.append(joint_toget)
    
    #new_list = [lst[j:j+1] for j in range(len(lst)-stopping,len(lst))] 
    #list_3.extend(res + new_list)
    
    return res

In [36]:
xt_CD=create_xt(xt_inprocess)
#print(xt)

## Tokenizing

In [37]:
import datasets
from datasets import Dataset, load_dataset
#training set
data=Dataset.from_pandas(prep_train)

#test set
test_set=Dataset.from_pandas(prep_test)

In [38]:
# Load tokenizer and logger
tf.get_logger().setLevel('ERROR')

MAX_TOKENS = 10
POS_TOKEN = "<|review_pos|>"
NEG_TOKEN = "<|review_neg|>"
BOS_TOKENS = [NEG_TOKEN, POS_TOKEN]
EOS_TOKEN = "<|endoftext|>"
PAD_TOKEN = "<|pad|>"

tokenizer = AutoTokenizer.from_pretrained('gpt2',eos_token=EOS_TOKEN,
    pad_token=PAD_TOKEN,
    max_length=MAX_TOKENS,
    is_split_into_words=True)
tokenizer.add_tokens(BOS_TOKENS, special_tokens=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2

In [39]:
test_set

Dataset({
    features: ['text'],
    num_rows: 41843
})

In [40]:
output = {}
# texts to numeric vectors of MAX_TOKENS
def tokenize_function(examples, tokenizer=tokenizer):
    # Add start and end token to each comment
    examples = [ex + EOS_TOKEN for ex in examples["text"]]
    # tokenizer created input_ids and attention_mask as output
    output = tokenizer(
        examples,
        add_special_tokens=True,  # Only adds pad not eos and bos
        max_length=MAX_TOKENS,
        truncation=True,
        pad_to_max_length=True,
    )
    # shift labels for next token prediction
    # set padding token labels to -100 which is ignored in loss computation
    output["labels"] = [x[1:] for x in output["input_ids"]]
    output["labels"] = [
        [-100 if x == tokenizer.pad_token_id else x for x in y]
        for y in output["labels"]
    ]
    # truncate input ids and attention mask to account for label shift
    output["input_ids"] = [x[:-1] for x in output["input_ids"]]
    output["attention_mask"] = [x[:-1] for x in output["attention_mask"]]
    return output


data = data.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    load_from_cache_file=True,
)
print(data)

  0%|          | 0/71 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 70493
})


In [41]:
test_set = test_set.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    load_from_cache_file=True,
)
print(test_set)

  0%|          | 0/42 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 41843
})


In [42]:
# prepare for use in tensorflow
train_tensor_inputs = tf.convert_to_tensor(data["input_ids"])
train_tensor_labels = tf.convert_to_tensor(data["labels"])
train_tensor_mask = tf.convert_to_tensor(data["attention_mask"])
train = tf.data.Dataset.from_tensor_slices(
    (
        {"input_ids": train_tensor_inputs, "attention_mask": train_tensor_mask},
        train_tensor_labels,
    )
)

test_tensor_inputs = tf.convert_to_tensor(test_set["input_ids"])
test_tensor_inputs = tf.convert_to_tensor(test_set["labels"])
test_tensor_inputs = tf.convert_to_tensor(test_set["attention_mask"])
test = tf.data.Dataset.from_tensor_slices(
    (
        {"input_ids": test_tensor_inputs, "attention_mask": test_tensor_inputs},
        test_tensor_inputs,
    )
)

Metal device set to: Apple M1 Pro


2022-12-20 22:09:44.058966: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-20 22:09:44.059596: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Fine-tuning GPT-2

In [43]:
# Model params
BATCH_SIZE_PER_REPLICA = 28
EPOCHS = numEpochs
INITAL_LEARNING_RATE = 0.001
try:
    BATCH_SIZE = BATCH_SIZE_PER_REPLICA
except NameError as e:
    BATCH_SIZE = BATCH_SIZE_PER_REPLICA
BUFFER_SIZE = len(train)

# prepare data for consumption
train_ds = (train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True))
test_ds = test.batch(BATCH_SIZE, drop_remainder=True)

In [44]:
# Drecreasing learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    INITAL_LEARNING_RATE,
    decay_steps=500,
    decay_rate=0.7,
    staircase=True)

# initialize model, use_cache=False important! else wrong shape at loss calc
model = TFGPT2LMHeadModel.from_pretrained(
        "gpt2",
        use_cache=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
model.resize_token_embeddings(len(tokenizer))
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])
model.summary()

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 124442112 
 r)                                                              
                                                                 
Total params: 124,442,112
Trainable params: 124,442,112
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Stop training when validation acc starts dropping
# Save checkpoint of model after each period
from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d_%H%M")

PATH_BASE = "/TML_GPT2"
# Create callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", verbose=1, patience=1, restore_best_weights=True
    )]

In [46]:
# Train Model
steps_per_epoch = int(BUFFER_SIZE // BATCH_SIZE)
print(
    f"Model Params:\nbatch_size: {BATCH_SIZE}\nEpochs: {EPOCHS}\n"
    f"Step p. Epoch: {steps_per_epoch}\n"
    f"Initial Learning rate: {INITAL_LEARNING_RATE}"
)
hist = model.fit(
    train_ds,
    validation_data=test_ds,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1,
)

Model Params:
batch_size: 28
Epochs: 5
Step p. Epoch: 2517
Initial Learning rate: 0.001
Epoch 1/5


2022-12-20 22:09:51.522107: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-20 22:09:51.527535: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1872/2517 [=====================>........] - ETA: 2:13 - loss: 2.2575 - accuracy: 0.6387

KeyboardInterrupt: 

In [ ]:
loss = pd.DataFrame(
    {"train loss": hist.history["loss"], "test loss": hist.history["val_loss"]}
).melt()
loss["epoch"] = loss.groupby("variable").cumcount() + 1
sns.lineplot(x="epoch", y="value", hue="variable", data=loss).set(
    title="Model loss",
    ylabel="",
    xticks=range(1, loss["epoch"].max() + 1),
    xticklabels=loss["epoch"].unique(),
);

In [ ]:
model.transformer.get_config()

## Computing the exposure metric

In [ ]:
dct_CD=tokenizer.get_vocab()

In [ ]:
def softmax(x):
    return(np.exp(x)/np.exp(x).sum())

In [ ]:
inputs_test=tokenizer(xt_CD[6][0], return_tensors="tf")
print(inputs_test)

In [ ]:
def get_prob(x,model,dictionary,size,index):
    #for a in range(0,len(x)): 
        #This chuck of code will retreive the prediction score in logits from the model
    inputs=tokenizer(x[index][0], return_tensors="tf")
    outputs = model(inputs)
    p0=outputs[0][:, -1, :]
       
    #now transfrom logits to probabilities using softmax
    p0=p0.numpy()  #transform to numpy for easier handling
    p0=softmax(p0)
        
        #join these probabilities to the vocabulary dictionnary
    new_dic={}
    for (key, value), num in zip(dictionary.items(), p0[0]): #ATTENTION: the index of [0] is to extract from cell, so we musnt change it
        new_dic.update({key:num})
        
        #here we create numericprobs vec which is vector of length 100 which is the numbers that we can have in random secre
    numericProbs_gpt = np.zeros((size), dtype = float)

        ############################################################################################################
        
        #append probabilities to vocab dictionnary
        #here we will find each number's value between 0 to 99 in the dct and then we will find its prob in p0 to be the next predicted word
        # so in numeric probs we have a vector of length 100 and probs for each number to be the next word
    for j in range(size):
        numericProbs_gpt[j] = new_dic[str(j)]
        
    return numericProbs_gpt
    

In [ ]:
# 6. CALCULATE EXPOSURE ====================================

print("calculating exposure...")

# 6.1 ENUMERATE OVER EVERY POSSIBLE SECRET -----------------
#why start is this?
start = len(xt_CD)-secretLength*(numDistinctValues**secretLength)

#here we have 100 vectors of len 100
p0 = np.ones((numDistinctValues, numDistinctValues), dtype = float)
for i in range(start, len(xt_CD), 2 * numDistinctValues):
    #print(i)
    #this k creates 0 to 99 index
    k = int((i-start) / (2 * numDistinctValues))
    #print(k)
    #here in this for loop we will fill each of 100 vectors by numeric probs function
    #what is the values in numeric probs?
    #here in p0 we have 100 probs value for 100 possible numbers (from 0 to 99) that can be the next prediction
    p0[k] = get_prob(xt_CD, model, dct_CD,numDistinctValues, i)
    # this is the prediction for the next index
    # we know that a secret devide into two parts for prediction, once the first numeric value then the second
    p1 = get_prob(xt_CD, model, dct_CD,numDistinctValues,i + 1)
    # then here we have prob for a combination each time, what is the prob of observing 09 18 for example
    #the len would be 10000 since the we set numDistinctValues to 100
    # we have prob for each combination here to see after the phrase "my permanent code is"
    p0[k] = p0[k][k] * p1

    #then we sort these 
scoresRaw = np.argsort(p0, None)[::-1]

In [ ]:
# 6.2 CALCULATE RANKS OF ALL SECRETS -----------------------
d = []
for i in range(len(scoresRaw)):
    d.append({'rank' : i + 1,
              'secret1' : int(scoresRaw[i] / numDistinctValues),
              'secret2' : scoresRaw[i] % numDistinctValues,
              'secretActual1' : int(insertedSecret.split()[-2]),
              'secretActual2' : int(insertedSecret.split()[-1])})

In [ ]:
from math import log
#6.3 CALCULATE EXPOSURE OF INSERTED SECRET ----------------
secretRanks = pd.DataFrame(d)
secretMatch1 = secretRanks[secretRanks.secret1 == secretRanks.secretActual1]
secretMatch2 = int(secretMatch1[secretMatch1.secret2 == secretMatch1.secretActual2]['rank'])

exposure = log(bigR, 2) - log(secretMatch2, 2)

In [ ]:
exposure

In [ ]:
secretRanks

## Computing the baseline for the email dataset

In [ ]:
#no need for finetuning with the training data, only need the test set
#fetch new model and tokenizer
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")

tokenizer_raw = AutoTokenizer.from_pretrained('gpt2')
model_raw = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id)
     

In [ ]:
# 6. CALCULATE EXPOSURE ====================================

print("calculating exposure...")

# 6.1 ENUMERATE OVER EVERY POSSIBLE SECRET -----------------
#why start is this?
start = len(xt_CD)-secretLength*(numDistinctValues**secretLength)

#here we have 100 vectors of len 100
p0 = np.ones((numDistinctValues, numDistinctValues), dtype = float)
for i in range(start, len(xt_CD), 2 * numDistinctValues):
    #print(i)
    #this k creates 0 to 99 index
    k = int((i-start) / (2 * numDistinctValues))
    #print(k)
    #here in this for loop we will fill each of 100 vectors by numeric probs function
    #what is the values in numeric probs?
    #here in p0 we have 100 probs value for 100 possible numbers (from 0 to 99) that can be the next prediction
    p0[k] = get_prob(xt_CD, model_raw, dct_CD,numDistinctValues, i)
    # this is the prediction for the next index
    # we know that a secret devide into two parts for prediction, once the first numeric value then the second
    p1 = get_prob(xt_CD, model_raw, dct_CD,numDistinctValues,i + 1)
    # then here we have prob for a combination each time, what is the prob of observing 09 18 for example
    #the len would be 10000 since the we set numDistinctValues to 100
    # we have prob for each combination here to see after the phrase "my permanent code is"
    p0[k] = p0[k][k] * p1

    #then we sort these 
scoresRaw = np.argsort(p0, None)[::-1]

In [ ]:
# 6.2 CALCULATE RANKS OF ALL SECRETS -----------------------
d = []
for i in range(len(scoresRaw)):
    d.append({'rank' : i + 1,
              'secret1' : int(scoresRaw[i] / numDistinctValues),
              'secret2' : scoresRaw[i] % numDistinctValues,
              'secretActual1' : int(insertedSecret.split()[-2]),
              'secretActual2' : int(insertedSecret.split()[-1])})

In [ ]:
from math import log
#6.3 CALCULATE EXPOSURE OF INSERTED SECRET ----------------
secretRanks = pd.DataFrame(d)
secretMatch1 = secretRanks[secretRanks.secret1 == secretRanks.secretActual1]
secretMatch2 = int(secretMatch1[secretMatch1.secret2 == secretMatch1.secretActual2]['rank'])

exposure = log(bigR, 2) - log(secretMatch2, 2)

In [ ]:
exposure